In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.widgets.text('incremental_flag', '0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')
print(incremental_flag)

In [0]:
df_src = spark.sql("""
select DISTINCT (Model_ID) AS Model_id, Model_Category from parquet.`abfss://silver@swap01storageaccount.dfs.core.windows.net/carsalesdata`
""")


df_src.display()

In [0]:
if spark.catalog.tableExists('bmw_cars_catalog.gold.model'):
    df_sink = spark.sql('''
    select dim_model_key, Model_id, Model_Category from parquet.`abfss://silver@swap01storageaccount.dfs.core.windows.net/carsalesdata`
    where 1 = 0 ''')
else:
    df_sink = spark.sql('''
    select 1 as dim_model_key, Model_id, Model_Category from parquet.`abfss://silver@swap01storageaccount.dfs.core.windows.net/carsalesdata`
    where 1 = 0 ''')



df_sink.display()

In [0]:
df_filter = df_src.join(df_sink, df_src['Model_id'] == df_sink['Model_id'], 'left').select(df_src['Model_id'], df_src['Model_Category'], df_sink['dim_model_key'])
                                                                                                                                                 
df_filter.display()

In [0]:
df_filter_old = df_filter.filter(col('dim_model_key').isNotNull())

display(df_filter_old)

In [0]:
df_filter_new = df_filter.filter(col('dim_model_key').isNull()).select(df_src['Model_id'], df_src['Model_Category'])

display(df_filter_new)